In [50]:
import pandas as pd

In [51]:
# sensitivity calculation
def calc_sensitivity(df, dataset):
    df = df.loc[df.dataset_type == dataset]
    true_pos = df.loc[df.correct_transcript == True, 'counts'].sum()
    false_neg = df.loc[df.correct_transcript == False, 'counts'].sum()
    
    sens = true_pos / (true_pos + false_neg)
    
    return sens

In [57]:
df = pd.read_csv('talon/simulated_Gm12878_talon_read_annot.tsv', sep='\t')

df['control_tid'] = df.read_name.str.split(pat='_', n=1, expand=True)[0]
df['talon_tid'] = df.annot_transcript_id.str.split(pat='.', n=1, expand=True)[0]
df.loc[df.annot_transcript_id.str.contains('ENST'), ['control_tid', 'talon_tid', 'annot_transcript_id']].head()
df.loc[df.annot_transcript_id.str.contains('ENST'), 'annot_transcript_id'].head()
df['correct_transcript'] = df.control_tid == df.talon_tid
df['dataset_type'] = 0
df.loc[df.dataset.str.contains('perf'), 'dataset_type'] = 'Perfect'
df.loc[~df.dataset.str.contains('perf'), 'dataset_type'] = 'Normal'

In [58]:
# how good are these bois supported
temp = df[['correct_transcript', 'transcript_novelty', 'dataset_type', 'read_name']].groupby(['correct_transcript', 'transcript_novelty', 'dataset_type']).count()
temp.reset_index(inplace=True)
temp.rename({'read_name':'counts'}, axis=1, inplace=True)

print(calc_sensitivity(temp, 'Normal'))
print(calc_sensitivity(temp, 'Perfect'))

0.4225947057181051
0.5553668260211552


In [60]:
temp.loc[temp.dataset_type == 'Perfect']

,correct_transcript,transcript_novelty,dataset_type,counts
1,False,Antisense,Perfect,4401
3,False,Genomic,Perfect,211261
5,False,ISM,Perfect,1154159
7,False,Intergenic,Perfect,5343
9,False,Known,Perfect,320289
11,False,NIC,Perfect,8331
13,False,NNC,Perfect,27823
15,True,Known,Perfect,2162855


In [64]:
print(temp.loc[temp.dataset_type=='Perfect','counts'].sum())
print(1154159/temp.loc[temp.dataset_type=='Perfect','counts'].sum())

3894462
0.29635903495784527


In [65]:
# total number of reads
print(temp.counts.sum())

6018225


In [66]:
# filter for NIC, NNC that pass filter
pass_list = pd.read_csv('talon/whitelist.csv', header=None, names=['gene_ID', 'transcript_ID'])
df = df.merge(pass_list, on=['transcript_ID', 'gene_ID'], how='inner')
df = df.loc[df.transcript_novelty.isin(['Known', 'NIC', 'NNC'])]

In [67]:
# how good are these bois supported
temp = df[['correct_transcript', 'transcript_novelty', 'dataset_type', 'read_name']].groupby(['correct_transcript', 'transcript_novelty', 'dataset_type']).count()
temp.reset_index(inplace=True)
temp.rename({'read_name':'counts'}, axis=1, inplace=True)

print(calc_sensitivity(temp, 'Normal'))
print(calc_sensitivity(temp, 'Perfect'))

0.8319314722600213
0.8628842256989668


In [68]:
# total number of reads 
print(temp.counts.sum())

3554172


In [69]:
6018225-3554172

2464053